In [ ]:
import pandas as pd
import os
import re


In [ ]:
# Import Data
cvs_fp = r'C:\Users\iya\Dev\CVM\CVS\2022'
cvs_tour_fn = r'sandag_cvs_tour.xlsx'
tnc_tour_fn = r'sandag_tnc_tour.xlsx'
cvs_df_fn = r'cv_20221102_sandag_draftfinal.xlsx'
db_fn = r'SANDAG 2022 CV DataBase & Dictionaires_03_03_2023.xlsx'

establishment_data = pd.read_excel(os.path.join(cvs_fp, db_fn), sheet_name='Establishment Data')
trip_data = pd.read_excel(os.path.join(cvs_fp, db_fn), sheet_name='Trip Data')
tour_data = pd.read_excel(os.path.join(cvs_fp, cvs_tour_fn), sheet_name='sandag_cvs_tour')
veh_data = pd.read_excel(os.path.join(cvs_fp, db_fn), sheet_name='Vehicle Data')

In [ ]:
indDict = {
    'Agriculture/Mining': 'Industrial',
    'Construction': 'Industrial',
    'Education/Other public services': 'Service/Gov/Office/FA',
    'Industrial/Utilities': 'Transport',
    'Info/FIRE/Professional services': 'Service/Gov/Office/FA',
    'Leisure/Accommodations and Food': 'Service/Gov/Office/FA',
    'Manufacturing': 'Industrial',
    'Medical/Health Services': 'Service/Gov/Office/FA',
    'Retail': 'Retail',
    'Transportation': 'Transport',
    'Wholesale': 'Wholesale'
}

model_weight_dict = {
    'FORD TRANSIT': 'DA',
    'DODGE SPRINTER': 'DA',
    'CHEVROLET C 4500': 'MHD',
    'FORD F450 SUPER DUTY FLATBED': 'LHD',
    'INTERNATIONAL BOX': 'MHD',
    'FREIGHTLINER BOX': 'MHD',
    'INTERNATIONAL STAKE': 'MHD',
    'PETERBILT TANKER': 'HHD',
    'KENWORTH M2': 'MHD',
    'KENWORTH TANKER': 'MHD',
    'ISUZU NPR': 'MHD',
    'CHEVROLET 3500 EXPRESS BOX TRUCK' :'LHD',
    'CHEVROLET EXPRESS 2500': 'DA',
    'NISSAN NV3500': 'DA',
    'CHEVROLET EXPRESS VAN 3500': 'DA',
    'GMC SAVANA': 'DA',
    'FORD E450 BOX TRUCK': 'DA',
    'CHEVROLET CARGO VAN': 'DA',
    'FREIGHTLINER COLUMBIA': 'HHD',
    'FREIGHTLINER CASCADIA': 'HHD',
    'KENWORTH T680': 'HHD',
    'FREIGHTLINER SPRINTER 2500': 'DA',
    'FORD MAVERICK': 'DA',
    'PETERBILT 388 DUMP TRUCK': 'HHD',
    'PETERBILT 389 DUMP TRUCK': 'HHD',
    'HYUNDAI IONIQ': 'DA',
    'CHEVROLET TAHOE': 'DA',
    'CHEVROLET HD SINGLE CAB PK': 'DA',
    'CHEVROLET Z71': 'DA'
}

def assignWeightClass_model(make, model):
    global model_weight_dict
    mm = str(make) + ' ' + str(model)
    mm = mm.strip().upper()
    if mm not in model_weight_dict.keys():
        print(mm)
    return model_weight_dict[mm]

def assignWeightClass(make, model, c, w):
    model = str(model)
    if model.strip().upper() in ['SPRINTER', 'SPLINTER', 'TRANSIT', \
                               'PROMASTER', 'PROMASTER VAN', 'PRO MASTER', 'SPRINTER 2500']:
        return 'LHD'

    if make.strip().upper() in ['TOYOTA', 'SUBARU','HONDA', 'BUICK', 'KIA', 'HYUNDAI'\
                                'CHRYSLER', 'MERCEDES BENZ']:
        return 'DA'

    if make.strip().upper() == 'FORD':
        num_str = re.findall(r'\d+', model)
        if len(num_str):
            ford_model_num = int(num_str[0])
            if ford_model_num in [150]: return 'DA'
            elif ford_model_num in[250, 350, 450, 550]: return 'LHD'
            elif ford_model_num in[650, 750]: return 'MHD'

    if make.strip().upper() in ['CHEVROLET', 'DODGE', 'GMC']:
        num_str = re.findall(r'\d+', model)
        if len(num_str):
            chev_model_num = int(num_str[0])
            if chev_model_num == 1500: return 'DA'
            elif chev_model_num in[2500,3500,4500, 5500]: return 'LHD'
            elif chev_model_num in[5500, 6500]: return 'MHD'
        if'SILVERADO' in model.strip().upper() or 'COLORADO' in model.strip().upper():
            return 'DA'

    if make.strip().upper() == 'NISSAN':
        if '2600' in model \
            or 'Hino' in model: return 'MHD'
        if 'nv2500' in model.lower() \
            or 'nv3500' in model.lower(): return 'LHD'
        else: return 'DA'

    if make.strip().upper() in ['ISUZU', 'HINO']:
        return 'MHD'

    if make.strip().upper() == 'PETERBILT':
        mn = re.findall(r'\d+', model)
        if len(mn):
            if int(mn[0]) in [337, 200, 310, 320, 330]: return "MHD"
            else: return 'HHD'

    if c in [1]:
        if model.strip().upper() in ['263', 'ALPHA 1']:
            return 'LHD'
        else: return 'DA'

    if c in [2, 3, 5]:
        if w < 8500: return 'DA'
        elif 8500 <= w < 14000: return 'LHD'
        elif 14000 <= w < 33000: return 'MHD'
        elif 33000 <= w < 120000: return 'HHD'
        elif w > 1000000: return assignWeightClass_model(make, model)

    if c in [6, 7, 8]:
        if w >= 33000: return 'HHD'
        else: return 'MHD'

    print(make, model, c, w)
        #if w > 5000000: return assignWeightClass_model(make, model)
        #else: return 'HHD'

In [ ]:
# Add Vehicle Classification
trip_data = trip_data.merge(
    veh_data[['vehicle_classification', 'id']],
    left_on='vehicle_id',
    right_on='id',
    validate='many_to_one',
    how='left'
    )

# Add Industry Group
trip_data = trip_data.merge(establishment_data[
    ['company_id', 'base_location_Industry Group']
    ], on='company_id', how='left')

# Exclude Buses
trip_data = trip_data[trip_data.vehicle_classification != 4]
tour_data = tour_data[tour_data.VehClass != 4]
veh_data = veh_data[veh_data.vehicle_classification != 4]


# Assign Tour IDs (unsorted):
tour_id = []
counter = 0
start, end = tour_data['TOUR_START'], tour_data['TOUR_END']

started, ended = False, False
for se in list(zip(start, end)):
    if se == (True, False):
        started, ended = True, False
        counter += 1
    elif se == (False, True):
        started, ended = False, True
    #if ended and not started: counter += 1
    tour_id.append(counter)

tour_data['tour_id'] = tour_id

# Add Industry Group
tour_data = tour_data.merge(establishment_data[
    ['company_id', 'base_location_Industry Group']
    ], on='company_id', how='left')

# CVM Industry Group
tour_data['industry_cvm'] = tour_data.IndName.apply(lambda x: indDict[x.strip()])
indNum_CVMIndName_dict = tour_data[[
    'base_location_Industry Group', 'industry_cvm'
]].groupby('base_location_Industry Group').max().to_dict()['industry_cvm']
trip_data['industry_cvm'] = trip_data['base_location_Industry Group'].apply(
    lambda x: indNum_CVMIndName_dict[x]
)

# Assign CVM Vehicle Weight Class
veh_data['weight_class_cvm'] = veh_data.apply(lambda x:
                                         assignWeightClass(
                                             x['veh_make_other'],
                                             x['vehicle_model'],
                                             x['vehicle_classification'],
                                             x['vehicle_weight']
                                             ),axis=1)

# Assign CVM Vehicle Weight Class to tour_data
tour_data = tour_data.merge(veh_data[['weight_class_cvm', 'id']],
                      left_on='vehicle_id',
                      right_on='id',
                      how='left')

tour_data['weighted_distance'] = tour_data.TRAVEL_DISTANCE.fillna(0)*tour_data['Most.Likely.Estimate.Weight.Factor']
tour_data['VehClass_IndName'] = tour_data.apply(
    lambda x: x['weight_class_cvm'][0]+x['industry_cvm'][0], axis=1
)

# Tour Starts
tour_data_starts = tour_data[tour_data.TOUR_START == True]

In [ ]:
# Employee Counts by Industry
employees = establishment_data[['base_location_Industry Group', 'no_of_emp_work']]
employees = employees.groupby('base_location_Industry Group').sum()
employees

In [ ]:
pd.options.display.float_format = '{:20,.2f}'.format

weight_class_category = pd.CategoricalDtype(
    categories=['DA', 'LHD', 'MHD', 'HHD'],
    ordered=True,
)

ind_category = pd.CategoricalDtype(
    categories=['Industrial', 'Wholesale', 'Retail', 'Service/Gov/Office/FA', 'Transport'],
    ordered=True,
)

In [ ]:
# Trip Counts by Industry
trip_1 = trip_data[trip_data.arrival_time.notna()]
trip_1 = trip_1[
    ['industry_cvm', 'Most Likely Estimate Weight Factor']
    ].groupby('industry_cvm').sum().reset_index()
trip_1 = trip_1.rename(columns={'Most Likely Estimate Weight Factor': 'Trips_Total'})
trip_1

# Trip by VehClass and Industry
trip_2 = tour_data[tour_data.TOUR_START != True]
trip_2 = trip_2.pivot_table(index='industry_cvm',
                             columns='weight_class_cvm',
                             values='Most.Likely.Estimate.Weight.Factor',
                             aggfunc='sum'
                             )
trips_counts = trip_1.merge(trip_2, on='industry_cvm')
trips_counts.set_index('industry_cvm', inplace=True)
trips_counts = trips_counts.reindex(["Industrial", "Wholesale", "Retail", "Service/Gov/Office/FA", "Transport"])
trips_counts = trips_counts[['Trips_Total','DA','LHD','MHD', 'HHD']]
trips_counts = trips_counts.rename_axis(None, axis=0)
trips_counts.rename(columns={'DA': 'Trip_DA',
                             'LHD': 'Trip_LHD',
                             'MHD': 'Trip_MHD',
                             'HHD': 'Trip_HHD'}, inplace=True)
trips_counts



In [ ]:
# Number of Tours by Industry
tours_count = tour_data_starts.groupby('industry_cvm')['Most.Likely.Estimate.Weight.Factor'].sum()
tours_count = pd.DataFrame(tours_count)
tours_count = tours_count.rename_axis(None, axis=0)
tours_count.rename(columns={'Most.Likely.Estimate.Weight.Factor': 'Tours_Total'}, inplace=True)
tours_count = tours_count.reindex(["Industrial", "Wholesale", "Retail", "Service/Gov/Office/FA", "Transport"])
tours_count

In [ ]:
# VMT by Industry
vmt_1 = tour_data[['weighted_distance', 'industry_cvm']].groupby('industry_cvm').sum().reset_index()

# VMT by Industry and VehClass
vmt_2 = tour_data.pivot_table(index='industry_cvm',
                      columns='weight_class_cvm',
                      values='weighted_distance',
                      aggfunc='sum')

vmt_counts = vmt_1.merge(vmt_2, on='industry_cvm')
vmt_counts.set_index('industry_cvm', inplace=True)
vmt_counts = vmt_counts.reindex(["Industrial", "Wholesale", "Retail", "Service/Gov/Office/FA", "Transport"])
vmt_counts = vmt_counts[['weighted_distance','DA','LHD','MHD', 'HHD']]
vmt_counts = vmt_counts.rename_axis(None, axis=0)
vmt_counts.rename(columns={'DA': 'VMT_DA',
                             'LHD': 'VMT_LHD',
                             'MHD': 'VMT_MHD',
                             'HHD': 'VMT_HHD',
                             'weighted_distance': 'VMT_Total'}, inplace=True)
vmt_counts

In [ ]:
# Table 15
table15 = tours_count.join(trips_counts, validate='one_to_one').join(vmt_counts, validate='one_to_one')
table15

In [ ]:
pd.options.display.float_format = '{:20,.5f}'.format


In [ ]:
# Figure 9. Tour Start Time by Industry
pd.options.display.float_format = '{:20,.5f}'.format

figure9 = pd.DataFrame(tour_data_starts.groupby(['industry_cvm', 'DEPART_TOD'])['Most.Likely.Estimate.Weight.Factor'].sum().reset_index())
figure9 = pd.crosstab(
    figure9.industry_cvm.astype(ind_category),
    figure9.DEPART_TOD,
    values=figure9['Most.Likely.Estimate.Weight.Factor'],
    aggfunc='sum',
    normalize='index')
figure9.rename_axis(None, axis=0, inplace=True)
figure9.rename_axis(None, axis=1, inplace=True)
figure9['Source'] = 'Survey'
figure9.rename(columns={'AM Peak\n6:00am-8:59am': 'AM (6am - 9am)',
                        'Early AM\n3:00am-5:59am': 'OE (3am - 6am)',
                        'Evening\n7:00pm-2:59am': 'OL (7pm - 3am)',
                        'Midday\n9:00am-3:29pm': 'MD (9am - 3:30pm)',
                        'PM Peak\n3:30pm-6:59pm': 'PM (3:30pm - 7pm)'}, inplace=True)
figure9 = figure9[['Source','OE (3am - 6am)','AM (6am - 9am)','MD (9am - 3:30pm)','PM (3:30pm - 7pm)', 'OL (7pm - 3am)']]
figure9

In [ ]:
# Figure 10. Vehicle Type by Industry
figure10 = pd.crosstab(
    tour_data_starts.industry_cvm.astype(ind_category),
    tour_data_starts.weight_class_cvm.astype(weight_class_category),
    values=tour_data_starts['Most.Likely.Estimate.Weight.Factor'],
    aggfunc='sum',
    normalize='index')
figure10.rename_axis(None, axis=0, inplace=True)
figure10.rename_axis(None, axis=1, inplace=True)
figure10.reset_index(inplace=True)
# #figure10 = figure10.T.reset_index()
IndName_Updated = [[i]*4 for i in figure10['index'].to_list()]
IndName_Updated = [x for xs in IndName_Updated for x in xs]
Type = ['DA', 'LHD', 'MHD', 'HHD']*5
Survey =[]
for i, j in zip(IndName_Updated, Type):
    Survey.append(figure10.loc[(figure10['index']==i)].loc[:, j].to_list()[0])
figure10 = pd.DataFrame({'Industry_Name':IndName_Updated,
              'Vehicle_Type': Type,
              'Survey': Survey})
figure10

In [ ]:
# Figure 12. Avg Trip Length by Vehicle Class and Industry
tour_data_wc = tour_data[tour_data.TRAVEL_DISTANCE.notna()]

sum = tour_data_wc[['VehClass_IndName','weighted_distance']].groupby('VehClass_IndName').sum().reset_index()
tripWeightSum = tour_data_wc[['VehClass_IndName','Most.Likely.Estimate.Weight.Factor']].groupby('VehClass_IndName').sum().reset_index()
sum = sum.merge(tripWeightSum, on='VehClass_IndName')
sum['CVS_Distance'] = sum['weighted_distance'] / sum['Most.Likely.Estimate.Weight.Factor']
figure12 = sum[['VehClass_IndName', 'CVS_Distance']]
figure12.set_index('VehClass_IndName', inplace=True)
figure12.rename_axis(None, axis=0, inplace=True)
figure12 = figure12.reindex(['DI', 'DW', 'DR', 'DS', 'DT',\
                  'LI', 'LW', 'LR', 'LS', 'LT', \
                  'MI', 'MW', 'MR', 'MS', 'MT',\
                  'HI', 'HW', 'HR', 'HS', 'HT'])

figure12

In [ ]:
# veh_data[['veh_make_other','vehicle_model','vehicle_year', \
#           'weight_class_cvm', 'vehicle_classification', 'vehicle_weight']].to_csv('check_vehClass.csv')
table15.to_csv('survey_table15.csv')
figure9.to_csv('survey_figure9.csv')
figure10.to_csv('survey_figure10.csv')
figure12.to_csv('survey_figure12.csv')

In [ ]:
veh_data.weight_class_cvm.value_counts()